In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# import data and delete unsued column
df = pd.read_csv('/content/missile_attacks_daily.csv')
df=df.drop(['source'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   time_start         710 non-null    object
 1   time_end           710 non-null    object
 2   model              710 non-null    object
 3   launch_place       385 non-null    object
 4   target             682 non-null    object
 5   destroyed_details  32 non-null     object
 6   carrier            158 non-null    object
 7   launched           710 non-null    int64 
 8   destroyed          710 non-null    int64 
dtypes: int64(2), object(7)
memory usage: 50.0+ KB


In [3]:
#Convert time to datetime type

df['time_start']= pd.to_datetime(df['time_start'])
df['time_end']= pd.to_datetime(df['time_end'])
df.head()

,time_start,time_end,model,launch_place,target,destroyed_details,carrier,launched,destroyed
0,2022-09-28 00:00:00,2022-09-28 00:00:00,X-59,NaN,Dnipropetrovsk oblast and Zaporizhzhia oblast,NaN,Su-35,5,4
1,2022-09-28 00:00:00,2022-09-28 00:00:00,Reconnaissance UAV,NaN,Ukraine,NaN,NaN,3,3
2,2022-09-29 23:00:00,2022-09-29 23:59:00,Shahed-136/131,NaN,Odesa oblast and Mykolaiv oblast,NaN,NaN,7,5
3,2022-09-30 00:00:00,2022-09-30 00:00:00,Orlan-10,NaN,Ukraine,NaN,NaN,3,3
4,2022-10-01 12:00:00,2022-10-01 12:00:00,Orlan-10,NaN,Ukraine,NaN,NaN,1,1


In [4]:
#remove duplicates
df = df.drop_duplicates()

In [5]:
#Create id
df["id"] = df.index + 1

<ipython-input-5-e2372a16b458>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["id"] = df.index + 1


In [6]:
#Creating separete dataframes for columns launch_place, target, destroyed, carrier, model

launch_place_df = df[['id', 'launch_place']].copy()
target_df = df[['id', 'target']].copy()
destroyed_details_df = df[['id', 'destroyed_details']].copy()
carrier_df = df[['id', 'carrier']].copy()
model_df =  df[['id', 'model']].copy()

df=df.drop(['launch_place'], axis=1)
df=df.drop(['target'], axis=1)
df=df.drop(['destroyed_details'], axis=1)
df=df.drop(['carrier'], axis=1)
df=df.drop(['model'], axis=1)

In [7]:
#Function to devide values in one column and remove it to separete raws
def split_and_duplicate_rows(df, column, delimiter):
    df_split = df[column].str.split(delimiter, expand=True)
    df_split = pd.DataFrame(df_split.stack()).reset_index(level=1, drop=True).rename(columns={0: column})
    df_merged = df_split.merge(df, left_index=True, right_index=True)


    df_merged.drop(column+'_y', axis=1, inplace=True)
    df_merged.reset_index(drop=True, inplace=True)
    df_merged.rename(columns={column+'_x': column}, inplace=True)

    return df_merged

In [8]:
carrier_df = split_and_duplicate_rows(carrier_df, 'carrier',' and ')
carrier_df['carrier'].unique()

array(['Su-35', '6 x Tu-22M3', '7 x Su-35', 'Tu-95MS', 'Tu-160', 'Navi',
       'Tu-95', '14 x Tu-95MS', '9 x Tu-95MS', '10 x Tu-95MS',
       '8 x Tu-95MS', 'Tu-22M3', 'Admiral Makarov', 'Iskander-M',
       '8 x TU-95MS', '4 x Tu-22M3', '1 x Tu-22M3', '6 x Tu-95MS',
       'MiG-31K', '2 x Tu-95MS', '3 x Navi', '7 x Tu-22M3', '6 x MiG-31K',
       '2 x Tu-160', '9 x Tu-95', '1 x MiG-31K', '4 x Tu-95MS',
       '8 x Tu-95', '8 x Tu-22M3', 'Submarines', 'Admiral Essen',
       'Ingushetiya', 'K-300P Bastion-P', '4 x MiG-31K', '11 x Tu-95MS',
       'Su-34', '5 x Tu-95MS', '2 x Su-34', '7 x Tu-95MS', '3 x MiG-31K',
       '18 x Tu-95MS', '5 x MiG-31K', '16 x Tu-95MS'], dtype=object)

In [9]:
#function to extract number of carriers
def extract_and_remove_number(s):
    if pd.isna(s):
        return 0, np.nan
    else:
        parts = s.split(' x ')
        if len(parts) == 2:
            number_of_carriers = int(parts[0])
            remaining_text = parts[1]
        else:
            number_of_carriers = 1
            remaining_text = s
        return number_of_carriers, remaining_text.strip()

In [10]:
# Extract number of carriers
carrier_df['number_of_carriers'], carrier_df['carrier'] = zip(*carrier_df['carrier'].map(extract_and_remove_number))
carrier_df

,carrier,id,number_of_carriers
0,Su-35,1,1
1,Tu-22M3,19,6
2,Su-35,20,7
3,Tu-95MS,22,1
4,Tu-160,22,1
...,...,...,...
170,Su-34,695,1
171,Tu-95MS,697,6
172,Tu-22M3,698,1
173,Su-34,700,1


In [11]:
#Function to get information about geagrafical place like coordinats
from geopy.geocoders import Nominatim

def get_location_info(place_name):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(place_name, language="en")

    if location:
      addresstype = location.raw.get('addresstype', 'N/A')
      if addresstype == 'city':
          return {
              'place':place_name,
              'city': place_name,
              'Oblast': location.address.split(",")[-2].strip(),
              'Country': location.address.split(",")[-1].strip(),
              'latitude': location.latitude,
              'longitude':location.longitude
          }
      elif addresstype == 'state':
        return {
              'place':place_name,
              'city': np.nan,
              'Oblast': location.address.split(",")[-2].strip(),
              'Country': location.address.split(",")[-1].strip(),
              'latitude': location.latitude,
              'longitude':location.longitude
          }
      elif addresstype == 'country':
          return {
              'place':place_name,
              'city': np.nan,
              'Oblast': np.nan,
              'Country': location.address.split(",")[-1].strip(),
              'latitude': location.latitude,
              'longitude':location.longitude
          }
      else:
          return {
              'place':place_name,
              'city': np.nan,
              'Oblast': np.nan,
              'Country': np.nan,
              'latitude': location.latitude,
              'longitude':location.longitude
          }
    else:
        return {
              'place':place_name,
              'city': np.nan,
              'Oblast': np.nan,
              'Country': np.nan,
              'latitude': np.nan,
              'longitude':np.nan
        }

In [12]:
#restructure multivalue columns
launch_place_df = split_and_duplicate_rows(launch_place_df, 'launch_place',' and ')
launch_place_df['launch_place'].unique()

array(['Ryazan oblast', 'Rostov oblast', 'Caspian Sea', 'Black Sea',
       'Kursk oblast', 'Belarus', 'north-east', 'Volgodonsk',
       'Zaporizhzhia oblast', 'south-east',
       'eastern coast of the Sea of Azov', 'Saratov', 'Sea of Azov',
       'east', 'north', 'Kursk obalst', 'Belgorod', 'Tokmak', 'Melitopol',
       'Bryansk oblast', 'Morozovsk', 'south', 'Olenegorsk', 'Tarkhankut',
       'Bryansk oblast, Seshcha', 'Crimea', 'Krasnodar Krai',
       'Primorsko-Akhtarsk', 'Dzhankoi, Crimea', 'Chauda, Crimea',
       'Tula oblast', 'Lipetsk oblast', 'Black Sea, Yalta', 'Soltsy-2',
       'Shaykovka', 'Engels-2', 'Olenya', 'Kherson oblast',
       'Black Sea, Feodosia', 'Yeysk', 'Volgograd oblast',
       'Sevastopol, Crimea', 'Belgorod oblast', 'Donetsk oblast',
       'Voronezh oblast', 'Saratov oblast', 'Belbek, Crimea',
       'Balaklava, Crimea', 'Savasleyka', 'Astrakhan oblast',
       'Primorsko-Akhtarske', 'Ryazan', 'Tambov', 'Luhansk oblast',
       'Oryol oblast'], dtyp

In [13]:
#Get information about launch place
launch_place_unique = launch_place_df['launch_place'].unique()
filter =  ['north','east','south','west','front line','center','north-east','south-east','eastern coast of the Sea of Azov']
launch_place_unique = [value for value in launch_place_unique if value not in filter]

unique_results = pd.DataFrame([get_location_info(place) for place in launch_place_unique])
launch_place_df = pd.merge(launch_place_df, unique_results, how='inner', left_on='launch_place', right_on='place')


In [14]:
launch_place_df

,launch_place,id,place,city,Oblast,Country,latitude,longitude
0,Ryazan oblast,19,Ryazan oblast,NaN,Central Federal District,Russia,54.422673,40.570525
1,Rostov oblast,20,Rostov oblast,NaN,Southern Federal District,Russia,47.622245,40.795794
2,Rostov oblast,287,Rostov oblast,NaN,Southern Federal District,Russia,47.622245,40.795794
3,Rostov oblast,707,Rostov oblast,NaN,Southern Federal District,Russia,47.622245,40.795794
4,Caspian Sea,22,Caspian Sea,NaN,NaN,NaN,41.778977,50.560758
...,...,...,...,...,...,...,...,...
402,Ryazan,662,Ryazan,Ryazan,Central Federal District,Russia,54.629569,39.742504
403,Tambov,662,Tambov,Tambov,Central Federal District,Russia,52.731118,41.434199
404,Tambov,671,Tambov,Tambov,Central Federal District,Russia,52.731118,41.434199
405,Luhansk oblast,692,Luhansk oblast,NaN,Luhansk Oblast,Ukraine,49.272459,38.915048


In [15]:
#restructure multivalue columns
target_df = split_and_duplicate_rows(target_df, 'target',' and ')
target_df['target'].unique()

array(['Dnipropetrovsk oblast', 'Zaporizhzhia oblast', 'Ukraine',
       'Odesa oblast', 'Mykolaiv oblast', 'south', 'Ochakiv',
       'Zaporizhzhia', 'Kyiv oblast', 'north', 'west', 'Kherson oblast',
       'Vinnytsia oblast', 'Poltava oblast', 'Donetsk oblast', 'east',
       'center', 'Dnipro', 'Kharkiv oblast', 'south-east', 'front line',
       'north-east', 'Kyiv oblast (south)', 'Cherkasy oblast',
       'Sumy oblast', 'Khmelnytskyi oblast', 'Lviv oblast',
       'Kirovohrad oblast', 'Kolomyia', 'Kryvyi Rih', 'Kharkiv',
       'Dnipropetrovsk oblast, Kryvyi Rih Raion',
       'Dnipropetrovsk oblast, Synelnykove Raion',
       'Dnipropetrovsk oblast, Kamianske Raion',
       'Dnipropetrovsk oblast, Dnipro Raion',
       'Poltava oblast, Myrhorod Raion',
       'Dnipropetrovsk oblast, Pavlohrad Raion', 'Kyiv',
       'Starokostiantyniv'], dtype=object)

In [16]:
#Get information about target place
target_place_unique = target_df['target'].unique()
filter =  ['north','east','south','west','front line','center','north-east','south-east']
target_place_unique = [value for value in target_place_unique if value not in filter]

unique_results = pd.DataFrame([get_location_info(place) for place in target_place_unique])
target_df = pd.merge(target_df, unique_results, how='inner', left_on='target', right_on='place')

In [17]:
target_df.drop('place', axis=1, inplace=True)

In [18]:
# Get info about structure of destroyed_details_df
destroyed_details_df['destroyed_details'].unique()


array([nan,
       "{'Mykolaiv oblast': 2, 'Kherson oblast': 1, 'Kharkiv oblast': 6}",
       "{'Odesa oblast': 9, 'Mykolaiv oblast': 7, 'Kherson oblast': 3, 'Kharkiv oblast': 5,    'Dnipropetrovsk oblast': 4, 'Zaporizhzhia oblast': 1}",
       "{'Odesa oblast': 11, 'Mykolaiv oblast': 3, Khmelnytskyi oblast: 1}",
       "{'Odesa oblast': 2, 'Mykolaiv oblast': 1, 'Dnipropetrovsk oblast': 1}",
       "{'Odesa oblast': 9}",
       "{'Odesa oblast': 32, 'Mykolaiv oblast': 5, 'Kherson oblast': 2}",
       "{'Mykolaiv oblast': 8, 'Kherson oblast': 2, 'Khmelnytskyi oblast': 3}",
       "{'Mykolaiv oblast': 2, 'Kherson oblast': 9}",
       "{'Odesa oblast': 9, 'Kherson oblast': 7,    'Khmelnytskyi oblast': 2, 'Zaporizhzhia oblast': 2 }",
       "{'Khmelnytskyi oblast': 2}",
       "{'Khmelnytskyi oblast': 2, 'Dnipropetrovsk oblast': 1,     'Odesa oblast': 4, 'Kherson oblast': 6}",
       "{'Khmelnytskyi oblast': 1, 'Dnipropetrovsk oblast': 2, 'Zaporizhzhia oblast': 1    'Mykolaiv oblast': 13, 

In [19]:
#Function to parse json in column as separeta rows
def process_json_string(json_string):
    try:
        return json.loads(json_string.replace("'", "\""))
    except (json.JSONDecodeError, AttributeError):
        return None

destroyed_details_df['place_dict'] = destroyed_details_df['destroyed_details'].apply(process_json_string)
destroyed_details_df = destroyed_details_df.dropna(subset=['place_dict'])

destroyed_details_df = pd.DataFrame(
    [(row['id'], key, value) for _, row in destroyed_details_df.iterrows() for key, value in row['place_dict'].items()],
    columns=['id', 'place', 'number']
)

In [20]:
destroyed_details_df

,id,place,number
0,567,Mykolaiv oblast,2
1,567,Kherson oblast,1
2,567,Kharkiv oblast,6
3,574,Odesa oblast,9
4,574,Mykolaiv oblast,7
...,...,...,...
86,691,Odesa oblast,1
87,691,Mykolaiv oblast,1
88,694,Dnipropetrovsk oblast,3
89,694,Kherson oblast,2


In [21]:
#get info about destruction place
destroyed_details_unique = destroyed_details_df['place'].unique()
filter =  ['north','east','south','west','front line','center','north-east']
destroyed_details_unique = [value for value in destroyed_details_unique if value not in filter]

unique_results = pd.DataFrame([get_location_info(place) for place in destroyed_details_unique])
destroyed_details_df = pd.merge(destroyed_details_df, unique_results, how='inner', left_on='place', right_on='place')

In [22]:
destroyed_details_df

,id,place,number,city,Oblast,Country,latitude,longitude
0,567,Mykolaiv oblast,2,NaN,Mykolaiv Oblast,Ukraine,47.388603,31.944233
1,574,Mykolaiv oblast,7,NaN,Mykolaiv Oblast,Ukraine,47.388603,31.944233
2,585,Mykolaiv oblast,1,NaN,Mykolaiv Oblast,Ukraine,47.388603,31.944233
3,588,Mykolaiv oblast,5,NaN,Mykolaiv Oblast,Ukraine,47.388603,31.944233
4,591,Mykolaiv oblast,8,NaN,Mykolaiv Oblast,Ukraine,47.388603,31.944233
...,...,...,...,...,...,...,...,...
81,691,Khmelnytskyi oblast,1,NaN,Khmelnytskyi Oblast,Ukraine,49.268624,27.063600
82,632,Kherson,1,NaN,Kherson Oblast,Ukraine,46.542172,33.407933
83,632,Zaporizhzhia oblast,3,NaN,NaN,NaN,47.961107,30.601522
84,663,Dnipropetrovsk oblast and Kharkiv oblast,6,NaN,NaN,NaN,NaN,NaN


In [23]:
#Restructure and get unique values in column
model_df = split_and_duplicate_rows(model_df, 'model',' and ')
model_df['model'].unique()

array(['X-59', 'Reconnaissance UAV', 'Shahed-136/131', 'Orlan-10',
       'Mohajer-6', 'Unknown UAV', 'X-22', 'C-300', 'X-101/X-555',
       'Kalibr', 'Lancet', 'Unknown Missile', 'Orion', 'X-31P', 'X-59MK2',
       'X-32', 'Merlin-VR', 'X-47 Kinzhal', 'X-31', 'P-800 Oniks', 'KAB',
       'Granat-4', 'Supercam', 'ZALA', 'C-400', 'Iskander-M',
       'Iskander-K', 'Orlan-30', 'X-35'], dtype=object)

In [25]:
#Export data to csv
df.to_csv('missile_attacks.csv', index=False)
destroyed_details_df.to_csv('missile_attacks_destroyed.csv', index=False)
launch_place_df.to_csv('missile_attacks_launch.csv', index=False)
target_df.to_csv('missile_attacks_target.csv', index=False)
model_df.to_csv('missile_attacks_model.csv', index=False)
carrier_df.to_csv('missile_attacks_carrier.csv', index=False)